# Generate YOLO Test Data

In [ ]:
import os
import glob
import json
import pprint
import re
from ultralytics import YOLO
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

# 

In [ ]:
modelre = re.compile(r"^(?P<model>11\w)_(?P<imgsize>\d+)imgsz_(?P<traindataset>[^_]+)_(?P<maxepochs>\d+)epochs")

dataset_dir="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/"
model_dir="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
result_dir="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/test"

datasets = [ "test-set-4cameras_yolov8",
             "test-set-4cameras-dayonly_yolov8",
             "test-set-4cameras-nightonly_yolov8",
             "test-set-1camera-ip8m-h-nw_yolov8", 
             "test-set-1camera-ip8m-h-sw_yolov8", 
             "test-set-1camera-ip8m-g-nw_yolov8", 
             "test-set-1camera-ip8m-g-sw_yolov8",
           ]
weights="best"
models = [ os.path.basename(d) for d in glob.glob(f"{model_dir}/11*epochs") ]
for modelname in models:
    print(modelname)
    m = modelre.match(modelname)
    if m is None:
        print(f"Couldn't figure out model")
        continue

    model_details = m.groupdict()
    model_details['name'] = modelname
    #{'imgsize': '1440',
    # 'maxepochs': '150',
    # 'model': '11s',
    # 'traindataset': 'IP8M-H-SW'}

    # load the model
    model = YOLO(os.path.join(model_dir, modelname, "weights", "{weights}.pt"), task="detect") 
    
    for d in datasets:
        print(f"{d}")
        os.chdir(os.path.join(dataset_dir, d))

        name=f"{modelname}_{weights}_{d}_test"
        if os.path.exists( os.path.join(result_dir, name, "metrics.json")):
            print(f"metrics.json exist for {modelname} / {d}, continuing")
            continue
            
        metrics = model.val(data=os.path.join(dataset_dir, d, "data.yaml"), 
                            project=result_dir, name=name, save_json=True, plots=True, 
                            imgsz=model_details["imgsize"], batch=1)
        store_metrics = metrics.results_dict.copy()
        store_metrics['category_mAP50-95'] = metrics.box.maps.tolist()
        with open(os.path.join(result_dir, name, "metrics.json"), "w") as o:
            o.write(json.dumps(store_metrics))